## Scikit-Image

Voici le notebook utilisant uniquement scikit-image. Il présente des cas d'utilisation utilisant les TP images. La version de scikit-image utilisée est la version 0.12. C'est la dernière version stable. Elle disponible via pip3. Il existe des différentes importantes entre les versions de scikit-images. L'API est en train de s'étendre.

On va commencer par charger une image et la transformée en niveau de gris.

In [ ]:
from skimage import io, color
img = color.rgb2gray(io.imread("tp1/talvi.jpg"))

Maintenant, contrairement aux autres bibliothèques, on va pouvoir calculer l'erreur quadratique simplement.

In [ ]:
from skimage import measure
def quadratique(img1, img2):
    if img1.shape == img2.shape:
        print(measure.compare_mse(img1, img2))
    else:
        print("Les images ne sont pas de la même taille")

Pour tester notre fonction ``quadratique``, nous allons faire une égalisation de l'image.

In [4]:
from skimage import exposure
img_egalisee = exposure.equalize_hist(img)
quadratique(img, img_egalisee)

0.0313464800077


Scikit-image permet de faire du seuillage. Mais d'après la documentation, il n'est pas possible de choisir un seuil arbitrairement.

Maintenant, nous allons afficher les deux images et leur histogramme pour concretement la différence. Une partie du code ci-dessous est extraite de la documentation de scikit-image.

In [ ]:
from skimage import exposure
import matplotlib.pyplot as plt
import numpy as np

def plot_img_and_hist(img, axes, bins=256):

    ax_img, ax_hist = axes
    ax_cdf = ax_hist.twinx()

    # Display image
    ax_img.imshow(img, cmap='gray')
    ax_img.set_axis_off()

    # Display histogram
    ax_hist.hist(img.ravel() * 255, bins=bins)
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
    ax_hist.set_xlabel('Pixel intensity')


    ax_hist.set_xlim(0, 255)
    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(img*255, bins)
    ax_cdf.plot(bins, img_cdf, 'r')

    return ax_img, ax_hist, ax_cdf


fig = plt.figure()
axes = np.zeros((2, 3), dtype=np.object)
axes[0,0] = plt.subplot(2, 2, 1, adjustable='box-forced')
axes[0,1] = plt.subplot(2, 2, 2, sharex=axes[0,0], sharey=axes[0,0], adjustable='box-forced')
axes[1,0] = plt.subplot(2, 2, 3)
axes[1,1] = plt.subplot(2, 2, 4)
ax_img, ax_hist, ax_cdf = plot_img_and_hist(img, axes[:, 0])
ax_img.set_title('Image originale')
ax_hist.set_ylabel('nombre de pixels')

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_egalisee, axes[:, 1])
ax_img.set_title('Image égalisée')

ax_cdf.set_ylabel("Part de l'intensité totale")

fig.subplots_adjust(wspace=0.4)
plt.show()

Continuons dans le TP1 avec l'agrandissement d'image. En quelques lignes, nous allons réaliser une agrandissement au plus proche voisin et bilinéaire

In [ ]:
from skimage import transform
def agrandissement(img):
    img_bilinear = transform.rescale(img, scale=2, mode='wrap', preserve_range=True, order=1)
    # order=1 <=> bi-linear (skimage.transform.warp)
    img_nearNeighbor = transform.rescale(image=img, scale=2, mode='wrap', preserve_range=True, order=0)
    # order=0 <=> nearest neighbor (skimage.transform.warp)

    fig = plt.figure()
    fig.add_subplot(1, 2, 1)
    plt.imshow(img_bilinear, cmap='gray')
    fig.add_subplot(1, 2, 2)
    plt.imshow(img_nearNeighbor, cmap='gray')
    plt.show()
agrandissement(img)

La partie sur la convolution du TP 2 peut être réaliser avec Numpy uniquement. Par contre, scikit-image devient intéressant dans la partie filtrage. Scikit-image permet de générer différents types de bruit notament le bruit "gaussien" et le bruit "poivre et sel".

In [ ]:
from scipy import ndimage
from skimage import filters, util
def filtre2D(img):
    N = 3
    type_bruit = 'AG'

    selem = np.ones([N, N])
    if type_bruit == 'AG':
        bruit = util.random_noise(np.zeros(img.shape), mode='gaussian')
        img_bruitee = util.random_noise(img, mode='gaussian')
    else:
        bruit = util.random_noise(np.zeros(img.shape), mode='s&p')
        img_bruitee = util.random_noise(img, mode='s&p')

    img_bruit_median = filters.median(img_bruitee, selem)
    img_bruit_linear = ndimage.convolve(img_bruitee, selem)

    fig = plt.figure()

    if type_bruit == 'AG':
        bruit_linear = ndimage.convolve(bruit, selem)
        img_linear = ndimage.convolve(img, selem)
        fig.add_subplot(3, 3, 1)
        plt.imshow(img, cmap='gray')
        fig.add_subplot(3, 3, 2)
        plt.imshow(bruit, cmap='gray')
        fig.add_subplot(3, 3, 3)
        plt.imshow(img_bruitee, cmap='gray')
        fig.add_subplot(3, 3, 4)
        plt.imshow(img_linear, cmap='gray')
        fig.add_subplot(3, 3, 5)
        plt.imshow(bruit_linear, cmap='gray')
        fig.add_subplot(3, 3, 6)
        plt.imshow(img_bruit_linear, cmap='gray')
        fig.add_subplot(3, 3, 9)
        plt.imshow(img_bruit_median, cmap='gray')
    else:
        fig.add_subplot(2, 2, 1)
        plt.imshow(img, cmap='gray')
        fig.add_subplot(2, 2, 2)
        plt.imshow(img_bruitee, cmap='gray')
        fig.add_subplot(2, 2, 3)
        plt.imshow(img_bruit_linear, cmap='gray')
        fig.add_subplot(2, 2, 4)
        plt.imshow(img_bruit_median, cmap='gray')
    plt.show()
filtre2D(img)

La partie "transformée de Fourier" du TP 3 peut être réaliser avec Numpy uniquement. Pour des raisons de temps, les parties "passe bas" et "morpho maths" n'ont pas été implémenté. Néanmoins, il existe des fonctions de l'API scikit-image permettant d'implémenter rapidement et simplement ces parties.